In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from model import micro, macro, params, distributions, calibrate
from importlib import reload
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Moments

In [2]:
moms_h = pd.read_pickle('moments/means_health.pkl')
moms_m = pd.read_pickle('moments/means_macro.pkl').transpose()
moms = moms_m.merge(moms_h,left_index=True,right_index=True,how='left')
moms = moms.loc['US',:]
moms = moms.rename({'gg':'trans_fromgood','gb':'trans_frombad','g_q2':'grad2','g_q3':'grad3','g_q4':'grad4'},axis=0)
moms = moms[['cshare','mshare','trans_fromgood','trans_frombad','grad2','grad3','grad4']]
moms

cshare            0.680889
mshare            0.144225
trans_fromgood    0.961600
trans_frombad     0.205977
grad2             1.177713
grad3             1.222356
grad4             1.276081
Name: US, dtype: float64

In [3]:
sd_h = pd.read_pickle('moments/stds_health.pkl')
sd_m = pd.read_pickle('moments/stds_macro.pkl').transpose()
sd = sd_m.merge(sd_h,left_index=True,right_index=True,how='left')
sd = sd.loc['US',:]
sd = sd.rename({'gg':'trans_fromgood','gb':'trans_frombad','g_q2':'grad2','g_q3':'grad3','g_q4':'grad4'},axis=0)
sd = sd[['cshare','mshare','trans_fromgood','trans_frombad','grad2','grad3','grad4']]
sd

cshare            0.009079
mshare            0.016178
trans_fromgood    0.001506
trans_frombad     0.009638
grad2             0.016617
grad3             0.016421
grad4             0.016628
Name: US, dtype: float64

In [4]:
moms = moms.to_frame().merge(sd.to_frame(),left_index=True,right_index=True)
moms.columns = ['mean','sd']
moms

,mean,sd
cshare,0.680889,0.009079
mshare,0.144225,0.016178
trans_fromgood,0.961600,0.001506
trans_frombad,0.205977,0.009638
grad2,1.177713,0.016617
grad3,1.222356,0.016421
grad4,1.276081,0.016628


# Initial Parameters

If need to start away from solution, try these:

In [5]:
guess_us = [ 2.1,  0.4,  0.15, -1,  3.5]

If want to start from solution, to check, try these:

In [6]:
guess_us = [ 2.105317188787775961600345908664,  0.397404995088832035943227083408,  0.161578046077853032524984655538, -0.988957955336655647471388874692,  3.511650634766380285611830913695]

In [7]:
ipars = calibrate.initpars(params.flexpars('us'))
ipars.fix('beta',0.97)
ipars.fix('price',1)
ipars.fix('tfp',1)
ipars.free('sigma',   guess_us[0])
ipars.free('phi',     guess_us[1])
ipars.free('psi',     guess_us[2])
ipars.free('delta_h1',guess_us[3])
ipars.free('delta_h2',guess_us[4])

In [8]:
ipars.print()

current parameter status : 
sigma 2.1053 True
beta 0.97 False
phi 0.3974 True
psi 0.1616 True
delta_h1 -0.989 True
delta_h2 3.5117 True
eta 0.0 False
tfp 1 False
price 1 False
risk 0.0 False


# Estimation

In [10]:
prob = calibrate.msm('us',initpar=ipars,nprocs=12)
prob.set_moments(moms)

-- 
number of parameters:  10
number of free parameters:  5
current parameter status : 
sigma 2.1053 True
beta 0.97 False
phi 0.3974 True
psi 0.1616 True
delta_h1 -0.989 True
delta_h2 3.5117 True
eta 0.0 False
tfp 1 False
price 1 False
risk 0.0 False
cshare 0.6808892149667144 0.009079433465468566
mshare 0.14422500000000002 0.01617846342973795
trans_fromgood 0.9615996486441867 0.0015062553539724164
trans_frombad 0.20597666978044457 0.009638163363181533
grad2 1.177712992763826 0.016617346336167793
grad3 1.2223556346247508 0.016421047162417955
grad4 1.2760806370630837 0.01662760246174447


In [12]:
prob.estimate()

KeyboardInterrupt: 

# Standard errors

In [ ]:
prob.covar()

In [ ]:
# Print and save the estimated parameters
print('preferences')
print('sigma =','{:.10f}'.format(prob.flex.sigma),'phi =','{:.10f}'.format(prob.flex.phi))
print('probability to be in good health')
print('psi =','{:.10f}'.format(prob.flex.psi),'delta_h0 =','{:.10f}'.format(prob.flex.delta[0]),'delta_h1 =','{:.10f}'.format(prob.flex.delta[1]))
epar_us = [ prob.flex.delta[0],  prob.flex.delta[1],  1         ,  1]
par_com = [  0.97,        prob.flex.psi,  prob.flex.sigma,  prob.flex.phi]
print(epar_us)
print(par_com)
print('se =',prob.covar.se)